---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [ ]:
def date_sorter():
    
    # Your code here
    #First variants of notation
    df1 = df.str.extractall(r'(?P<full>(?P<month>\d{1,2})[-/](?P<day>([0-2]?[0-9])|([3][01]))[-/](?P<year>\d{2,4}))')
    dates = df1.drop([3, 4], axis=1)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Second variants of notation
    df2 = df[indexes_left].str.extractall(r'(?P<full>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.-]?[ ]?(?P<day>\d{1,2})[,]?[- ]?(?P<year>\d{4}))')
    dates = dates.append(df2)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Third variant of notation
    df3 = df[indexes_left].str.extractall(r'(?P<full>(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]? (?P<year>\d{4}))')
    dates = dates.append(df3)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Fourth varaint od notation
    df4 = df[indexes_left].str.extractall(r'(?P<full>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]? (?P<day>\d{1,2}\w{2}), (?P<year>\d{2,4}))')
    dates = dates.append(df4)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Fifth variant of notation
    df5 = df[indexes_left].str.extractall(r'(?P<full>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]? (?P<year>\d{2,4}))')
    dates = dates.append(df5)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Sixth variant of notation
    df6 = df[indexes_left].str.extractall(r'(?P<full>(?P<month>\d{1,2})/(?P<year>\d{2,4}))')
    dates = dates.append(df6)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Seventh variant of notation
    df7 = df[indexes_left].str.extractall(r'(?P<full>(?P<year>\d{4}))')
    dates = dates.append(df7)
    indexes_left = ~df.index.isin([x[0] for x in dates.index])
    
    #Filling NaN
    dates = dates.fillna(1)
    
    #Notation unifying
    dates['year'] = dates['year'].apply(lambda x: str(x)).apply(lambda x: '19' + x if len(x) == 2 else x)
    dates['month'] = dates['month'].apply(lambda x: str(x)).apply(lambda x: '0' + x if len(x) == 1 else x)
    dates['day'] = dates['day'].apply(lambda x: str(x)).apply(lambda x: '0' + x if len(x) == 1 else x)
    
    months = dict({'Jan': '01', 'January': '01', 'Feb': '02', 'February': '02', 'Mar': '03', 'March': '03', 'Apr': '04',
               'April': '04', 'May': '05', 'Jun': '06', 'June': '06', 'Jul': '07', 'July': '07', 'Aug': '08', 'August': '08', 
              'Sep': '09', 'September': '09', 'Oct': '10', 'October': '10', 'Nov': '11', 'November': '11', 
              'Dec': '12', 'December': '12'})

    dates.replace({"month": months}, inplace=True)
    
    #Full converted dates to timestamps
    dates['date'] = dates['month'] + '/' + dates['day'] + '/' + dates['year']
    dates['date'] = pd.to_datetime(dates['date'])
    dates = dates.reset_index(drop=True)['date']
    
    #Sorted dates
    sorted_dates = pd.Series(dates.sort_values().index)
    
    
    return sorted_dates

In [ ]:
date_sorter();